<a href="https://colab.research.google.com/github/anitayadav3/EmotionRecognitionInConversation/blob/master/Practice_Workspace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/My Drive/DialogueRNN

/content/gdrive/My Drive/DialogueRNN


In [3]:
!python train_IEMOCAP.py --dropout 0.4 --lr 0.0003 --batch-size 32 --class-weight --l2 0.0

Namespace(active_listener=False, attention='general', batch_size=32, class_weight=True, dropout=0.4, epochs=60, l2=0.0, lr=0.0003, no_cuda=False, rec_dropout=0.1, tensorboard=False)
Running on GPU
epoch 1 train_loss 1.6546 train_acc 36.51 train_fscore35.66 valid_loss nan valid_acc nan val_fscorenan test_loss 1.4987 test_acc 55.21 test_fscore 50.33 time 7.58
epoch 2 train_loss 1.2016 train_acc 59.66 train_fscore57.89 valid_loss nan valid_acc nan val_fscorenan test_loss 1.1435 test_acc 60.87 test_fscore 57.66 time 7.49
epoch 3 train_loss 0.9009 train_acc 66.52 train_fscore65.38 valid_loss nan valid_acc nan val_fscorenan test_loss 0.9553 test_acc 65.0 test_fscore 63.64 time 7.42
epoch 4 train_loss 0.7576 train_acc 71.0 train_fscore70.63 valid_loss nan valid_acc nan val_fscorenan test_loss 0.9234 test_acc 63.46 test_fscore 62.41 time 7.11
epoch 5 train_loss 0.6671 train_acc 75.63 train_fscore75.46 valid_loss nan valid_acc nan val_fscorenan test_loss 0.9732 test_acc 62.66 test_fscore 62.31 

**Train_IEMOCAP file opened for practise**

In [4]:
import numpy as np
np.random.seed(1234)
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.optim as optim
import argparse
import time
import pickle
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score,\
                        classification_report, precision_recall_fscore_support
from model import BiModel, Model, MaskedNLLLoss
from dataloader import IEMOCAPDataset

In [5]:
def get_train_valid_sampler(trainset, valid=0.1):
    size = len(trainset)
    idx = list(range(size))
    split = int(valid*size)
    return SubsetRandomSampler(idx[split:]), SubsetRandomSampler(idx[:split])

def get_IEMOCAP_loaders(path, batch_size=32, valid=0.1, num_workers=0, pin_memory=False):
    trainset = IEMOCAPDataset(path=path)
    train_sampler, valid_sampler = get_train_valid_sampler(trainset, valid)
    train_loader = DataLoader(trainset,
                              batch_size=batch_size,
                              sampler=train_sampler,
                              collate_fn=trainset.collate_fn,
                              num_workers=num_workers,
                              pin_memory=pin_memory)
    valid_loader = DataLoader(trainset,
                              batch_size=batch_size,
                              sampler=valid_sampler,
                              collate_fn=trainset.collate_fn,
                              num_workers=num_workers,
                              pin_memory=pin_memory)

    testset = IEMOCAPDataset(path=path, train=False)
    test_loader = DataLoader(testset,
                             batch_size=batch_size,
                             collate_fn=testset.collate_fn,
                             num_workers=num_workers,
                             pin_memory=pin_memory)

    return train_loader, valid_loader, test_loader

def train_or_eval_model(model, loss_function, dataloader, epoch, optimizer=None, train=False):
    losses = []
    preds = []
    labels = []
    masks = []
    alphas, alphas_f, alphas_b, vids = [], [], [], []
    assert not train or optimizer!=None
    if train:
        model.train()
    else:
        model.eval()
    for data in dataloader:
        if train:
            optimizer.zero_grad()
        # import ipdb;ipdb.set_trace()
        textf, visuf, acouf, qmask, umask, label =\
                [d.cuda() for d in data[:-1]] if cuda else data[:-1]
        #log_prob = model(torch.cat((textf,acouf,visuf),dim=-1), qmask,umask,att2=True) # seq_len, batch, n_classes
        log_prob, alpha, alpha_f, alpha_b = model(textf, qmask,umask,att2=True) # seq_len, batch, n_classes
        lp_ = log_prob.transpose(0,1).contiguous().view(-1,log_prob.size()[2]) # batch*seq_len, n_classes
        labels_ = label.view(-1) # batch*seq_len
        loss = loss_function(lp_, labels_, umask)

        pred_ = torch.argmax(lp_,1) # batch*seq_len
        preds.append(pred_.data.cpu().numpy())
        labels.append(labels_.data.cpu().numpy())
        masks.append(umask.view(-1).cpu().numpy())

        losses.append(loss.item()*masks[-1].sum())
        if train:
            loss.backward()
            if args.tensorboard:
                for param in model.named_parameters():
                    writer.add_histogram(param[0], param[1].grad, epoch)
            optimizer.step()
        else:
            alphas += alpha
            alphas_f += alpha_f
            alphas_b += alpha_b
            vids += data[-1]

    if preds!=[]:
        preds  = np.concatenate(preds)
        labels = np.concatenate(labels)
        masks  = np.concatenate(masks)
    else:
        return float('nan'), float('nan'), [], [], [], float('nan'),[]

    avg_loss = round(np.sum(losses)/np.sum(masks),4)
    avg_accuracy = round(accuracy_score(labels,preds,sample_weight=masks)*100,2)
    avg_fscore = round(f1_score(labels,preds,sample_weight=masks,average='weighted')*100,2)
    return avg_loss, avg_accuracy, labels, preds, masks,avg_fscore, [alphas, alphas_f, alphas_b, vids]